# ATP Synthase Model Attempt 2
7.8.2020<br>
In this notebook, we will try to implement a control system for H+ concentration.

In [3]:
# Import cell (refined)
import numpy as np
import pandas as pd
import scipy.special as sp
import scipy.optimize
from scipy.optimize import curve_fit
import scipy.integrate
from scipy.integrate import odeint

import biocircuits

import bokeh.io
import bokeh.plotting
from bokeh.layouts import row

bokeh.io.output_notebook()



Loading BokehJS ...

## Premature optimization is root of all evil!!!!!

In [74]:
def rhs(var, t, k1, k2, k3):
    
    atp,adp,pi,h,h20,outside_H = var
    
    
    # PH CONTROL PORTION WHEN PH IS GREATER THAN 8, ATP SYNTHASE RUNS BACKWARDS (assume has same rxn rate:()
    ph = np.log10(outside_H)
    if ph > 8:
        #print(-np.log(outside_H))
        
        datp_dt = -k1 * atp * h**4
    
        dadp_dt = k1 * atp * h**4
    
        dpi_dt = k1 * atp * h**4
    
        dh_dt = -k3 * h
        
        dh20_dt = 0

        doutsideH_dt = H_control(var, t, k3)
        print("HERE")
        #print('heree', doutsideH_dt)
        
    else:
        #print(-np.log10(outside_H))
        #print("YEYA")
        datp_dt = k1 * adp * pi * h**3.3 - k2 * atp

        dadp_dt = -k1 * adp * pi * h**3.3 + k2 * atp

        dpi_dt = dadp_dt

        dh_dt = k3 * outside_H - k1 * adp * pi * h**4

        dh20_dt = k1 * adp * pi * h**4

        doutsideH_dt = H_control(var, t, k3)
    
    return np.array([datp_dt, dadp_dt, dpi_dt, dh_dt, dh20_dt, doutsideH_dt])
    


In [116]:
def H_control(var, t, k3):
    
    atp,adp,pi,h,h20,outside_H = var
    
    if t < 2:
        N = 0

    elif t > 2 and t <= 2.2:
        N = 30
    
    elif t > 20 and t <= 20.4:
        N = 30
        
    else:
        N = -k3 * outside_H
    
    return np.sin( 0.2*t)

In [117]:
def rhs_noatpsyn(var,t,k2):
    
    atp,adp,pi = var
    
    datp_dt = -k2 * atp
    
    dadp_dt = k2 * atp
    
    dpi_dt = k2 * atp
    
    return np.array([datp_dt, dadp_dt, dpi_dt])

Some rate justifications: 
- k1 = ATP synthesis rate is > 100 ATP/sec (bionum ID 111975) or 6,000 ATP/min or 360,000 ATP/hr <br>
- k2 = ATP Use rate in Tx/Tl? - we normally assume there is none by hour 20<br>
- k3 = Each ATP requires the transport or 3.33 protons, we round to 4. <br>
- what is k3: outside H to H rate: if 3.33 h+ atp/second and 100 atp/s then 400 h+/sec or 24000 h+ / min or 1,440,000 h+ /hr

In [118]:
t_vals = np.linspace(0,75,201)
k1 = 100 * 60 * 60 # per sec to hr
k2 = 100 * 60 * 60
k3 = 333 * 60 * 60 # per sec to hr

k1 = 0.1 # per sec to hr
k2 = 0.2
k3 = 0.1 # per sec to hr

x_args = (k1, k2, k3)

atp_0 = 30
adp_0 = 0
pi_0 = 0
h_0 = 0
h20_0 = 0 
outside_H = 1.5
y0 = np.array([atp_0, adp_0, pi_0, h_0, h20_0, outside_H])

x = odeint(rhs, y0, t_vals, args = x_args)

colors = ['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02']



p = bokeh.plotting.figure(
        frame_width=450,
        frame_height=200,
        x_axis_label="time (hrs)",
        y_axis_label="concs (mM)",
    title = 'All Species'
        )
p1 = bokeh.plotting.figure(
        frame_width=450,
        frame_height=200,
        x_axis_label="time (hrs)",
        y_axis_label="concs (mM)",title = 'Outside H+'
        )

p2 = bokeh.plotting.figure(
        frame_width=450,
        frame_height=200,
        x_axis_label="time (hrs)",
        y_axis_label="concs (mM)",title = 'No ATP Synthase'
        )

spec_list = ['atp', 'adp', 'pi', 'h+', 'h20', 'outside h+']

for i,spec in enumerate(spec_list):
    if spec_list[i] == 'h20':
        i+=1
    else:
        p.line(t_vals, x[:,i], line_width = 2, color = colors[i], legend_label = spec)
        

p1.line(t_vals, x[:,5], line_width = 2, color= colors[5] )
        
        
p.legend.click_policy = 'hide'


# Make a plot for use without the ATP synthase
noatpsyn_spec_list = ['atp', 'adp', 'pi']
noatpsyn_0 = np.array([atp_0, adp_0, pi_0])
noatpsyn_args = (k2,)

noatp = odeint(rhs_noatpsyn, noatpsyn_0, t_vals, args = noatpsyn_args)

for i,spec in enumerate(noatpsyn_spec_list):
    p2.line(t_vals, noatp[:,i], line_width = 2, color = colors[i], legend_label = spec)

bokeh.io.show(row(p, p1))
bokeh.io.show(p2)



Reduce model to just include outside H and atp and then perform linear stability 

In [84]:
# imports
from libsbml import *
import sys
import numpy as np
from auto_reduce import *
from auto_reduce.utils import get_ODE
import matplotlib.pyplot as plt
from auto_reduce.utils import reduce

In [85]:
# set num species and params
n = 6
num_params = 3

k1 = 0.1 # per sec to hr
k2 = 0.2
k3 = 0.1 # per sec to hr


P = np.zeros(num_params)
P[0] = k1
P[1] = k2
P[2] = k3

params_values = P.copy()
timepoints_ode = np.linspace(0,72,100)

atp_0 = 30
adp_0 = 0
pi_0 = 0
h_0 = 0
h20_0 = 0 
outside_H = 1.5
x_init = np.array([atp_0, adp_0, pi_0, h_0, h20_0, outside_H])

error_tol = 100
nstates_tol = 1

x,f,P = system.load_ODE_model(n, num_params)
params = P



In [88]:
x

[x0, x1, x2, x3, x4, x5]

In [87]:
#x[0] = atp
#x[1] = adp
#x[2] = pi
#x[3] = h
#x[4] = h20
#x[5] = outsideH

f[0] = P[0] * x[1] * x[2] * x[3]**4 - P[2] * x[0] # atp
f[1] = -P[0] * x[1] * x[2] * x[3]**4 + P[1] * x[0] # adp
f[2] = -P[0] * x[1] * x[2] * x[3]**4 + P[1] * x[0] # pi
f[3] = P[2] * x[5] - P[0] * x[1] * x[3]**4 # h
f[4] = P[0] * x[1] * x[2] * x[3]**4 #h20
f[5] = 0.2 * sin(t)


P0